In [1]:
import torch
import torch.nn as nn

In [15]:
class local_planar_guidance(nn.Module):
    def __init__(self, upratio):
        super(local_planar_guidance, self).__init__()
        self.upratio = upratio
        self.u = torch.arange(self.upratio).reshape([1, 1, self.upratio]).float()
        self.v = torch.arange(int(self.upratio)).reshape([1, self.upratio, 1]).float()
        self.upratio = float(upratio)

    def forward(self, plane_eq, focal):
        plane_eq_expanded = torch.repeat_interleave(plane_eq, int(self.upratio), 2)
        plane_eq_expanded = torch.repeat_interleave(plane_eq_expanded, int(self.upratio), 3)
        n1 = plane_eq_expanded[:, 0, :, :]
        n2 = plane_eq_expanded[:, 1, :, :]
        n3 = plane_eq_expanded[:, 2, :, :]
        n4 = plane_eq_expanded[:, 3, :, :]
        print('n1:', n1.shape)
        
        u = self.u.repeat(plane_eq.size(0), plane_eq.size(2) * int(self.upratio), plane_eq.size(3))
        u = (u - (self.upratio - 1) * 0.5) / self.upratio
        print('u: ',u.shape)
        
        v = self.v.repeat(plane_eq.size(0), plane_eq.size(2), plane_eq.size(3) * int(self.upratio))
        v = (v - (self.upratio - 1) * 0.5) / self.upratio
        print('v: ',v.shape)
        
        print('n4: ',n4.shape)
        return n4 / (n1 * u + n2 * v + n3)

In [23]:
lpg = local_planar_guidance(4)

In [24]:
input = torch.rand((3, 4, 56, 112))

In [25]:
input.shape

torch.Size([3, 4, 56, 112])

In [26]:
out = lpg(input, 1)

n1: torch.Size([3, 224, 448])
u:  torch.Size([3, 224, 448])
v:  torch.Size([3, 224, 448])
n4:  torch.Size([3, 224, 448])


In [27]:
out.shape

torch.Size([3, 224, 448])

In [28]:
out

tensor([[[ 6.0305e-01,  5.7900e-01,  5.5680e-01,  ..., -2.6612e+00,
          -9.9128e+00,  5.7467e+00],
         [ 5.3989e-01,  5.2054e-01,  5.0252e-01,  ..., -5.5489e+00,
           1.0562e+01,  2.7058e+00],
         [ 4.8871e-01,  4.7280e-01,  4.5789e-01,  ...,  6.5174e+01,
           3.4455e+00,  1.7695e+00],
         ...,
         [ 1.3916e+01,  1.1080e+01,  9.2040e+00,  ...,  1.0476e+00,
           1.0138e+00,  9.8218e-01],
         [ 6.9300e+00,  6.1465e+00,  5.5221e+00,  ...,  9.2314e-01,
           8.9684e-01,  8.7200e-01],
         [ 4.6138e+00,  4.2528e+00,  3.9443e+00,  ...,  8.2514e-01,
           8.0406e-01,  7.8404e-01]],

        [[ 1.3931e+00,  1.2106e+00,  1.0703e+00,  ..., -2.7406e+01,
           5.2148e+00,  2.3809e+00],
         [ 1.2493e+00,  1.1005e+00,  9.8339e-01,  ...,  5.0985e+00,
           2.3563e+00,  1.5322e+00],
         [ 1.1325e+00,  1.0088e+00,  9.0952e-01,  ...,  2.3323e+00,
           1.5220e+00,  1.1296e+00],
         ...,
         [ 7.9007e-01,  6

In [23]:
torch.mean(out)

tensor(-1.4884)